pyspark dataframe

In [0]:
from pyspark.sql import functions as F

data = [
    (1001, "C001", "Laptop", 1, 899.99, "2023_04_05"),
    (1002, "C002", "Mouse", 2, 25.00, "2023_04_06"),
    (1003, "C001", "Monitor", 1, 199.99, "2023_04_07"),
    (1004, "C003", "Keyboard", 1, 45.00, "2023_04_08"),
    (1005, "C002", "USB Cable", 3, 5.99, "2023_04_09")
]

cols = [
    "order_id",
    "customer_id",
    "product",
    "quantity",
    "unit_price",
    "order_dt"
]

df_orders = spark.createDataFrame(data, cols)

df_orders.show()


+--------+-----------+---------+--------+----------+----------+
|order_id|customer_id|  product|quantity|unit_price|  order_dt|
+--------+-----------+---------+--------+----------+----------+
|    1001|       C001|   Laptop|       1|    899.99|2023_04_05|
|    1002|       C002|    Mouse|       2|      25.0|2023_04_06|
|    1003|       C001|  Monitor|       1|    199.99|2023_04_07|
|    1004|       C003| Keyboard|       1|      45.0|2023_04_08|
|    1005|       C002|USB Cable|       3|      5.99|2023_04_09|
+--------+-----------+---------+--------+----------+----------+



2. Rename order_dt column to order_date and convert the column datatype to date

In [0]:
from pyspark.sql.functions import to_date

df = df.withColumnRenamed("order_dt", "order_date")
df = df.withColumn("order_date", to_date("order_date", "yyyy_MM_dd"))
df.show()


+--------+-----------+---------+--------+----------+----------+
|order_id|customer_id|  product|quantity|unit_price|order_date|
+--------+-----------+---------+--------+----------+----------+
|    1001|       C001|   Laptop|       1|    899.99|2023-04-05|
|    1002|       C002|    Mouse|       2|      25.0|2023-04-06|
|    1003|       C001|  Monitor|       1|    199.99|2023-04-07|
|    1004|       C003| Keyboard|       1|      45.0|2023-04-08|
|    1005|       C002|USB Cable|       3|      5.99|2023-04-09|
+--------+-----------+---------+--------+----------+----------+



In [0]:
df.createOrReplaceTempView("df")
display(spark.sql("SELECT * FROM df"))

order_id,customer_id,product,quantity,unit_price,order_dt
1001,C001,Laptop,1,899.99,2023_04_05
1002,C002,Mouse,2,25.0,2023_04_06
1003,C001,Monitor,1,199.99,2023_04_07
1004,C003,Keyboard,1,45.0,2023_04_08
1005,C002,USB Cable,3,5.99,2023_04_09


3. Find Customers who has purchased more than one order and the products they purchased. Output should be customer_id and their purchased products as a list

In [0]:
%sql


In [0]:
from pyspark.sql.functions import collect_list, count

df_grouped = df.groupBy("customer_id") \
    .agg(
        count("order_id").alias("order_count"),
        collect_list("product").alias("products")
    )

df_result = df_grouped.filter(df_grouped.order_count > 1)
df_result.select("customer_id", "products").show()


+-----------+------------------+
|customer_id|          products|
+-----------+------------------+
|       C001| [Laptop, Monitor]|
|       C002|[Mouse, USB Cable]|
+-----------+------------------+



4. Find the most recent order placed by each customer.

cols = ["order_id","customer_id","product","quantity","unit_price","order_date"]

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col

windowSpec = Window.partitionBy("customer_id").orderBy(col("order_date").desc())

df_with_rownum = df.withColumn("row_num", row_number().over(windowSpec))

df_latest = df_with_rownum.filter(df_with_rownum.row_num == 1)

df_latest.select("customer_id", "product", "order_date").show()


+-----------+---------+----------+
|customer_id|  product|order_date|
+-----------+---------+----------+
|       C001|  Monitor|2023-04-07|
|       C002|USB Cable|2023-04-09|
|       C003| Keyboard|2023-04-08|
+-----------+---------+----------+



Create a view of orders table from (1) and Write SQL queries to solve (2), (3) and (4) questions.

In [0]:
df.createOrReplaceTempView("orders")


In [0]:
display(spark.sql("select * from orders"))

order_id,customer_id,product,quantity,unit_price,order_date
1001,C001,Laptop,1,899.99,2023-04-05
1002,C002,Mouse,2,25.0,2023-04-06
1003,C001,Monitor,1,199.99,2023-04-07
1004,C003,Keyboard,1,45.0,2023-04-08
1005,C002,USB Cable,3,5.99,2023-04-09


In [0]:
spark.sql("""

SELECT
customer_id,
collect_list(product) as products

FROM orders

GROUP BY customer_id

HAVING COUNT(order_id) > 1

""").show()


+-----------+------------------+
|customer_id|          products|
+-----------+------------------+
|       C001| [Laptop, Monitor]|
|       C002|[Mouse, USB Cable]|
+-----------+------------------+



In [0]:
spark.sql("""

WITH latest_orders AS (

SELECT *,
ROW_NUMBER() OVER (
PARTITION BY customer_id
ORDER BY order_date DESC
) as rn

FROM orders

)

SELECT
customer_id,
product,
order_date

FROM latest_orders

WHERE rn = 1

""").show()


+-----------+---------+----------+
|customer_id|  product|order_date|
+-----------+---------+----------+
|       C001|  Monitor|2023-04-07|
|       C002|USB Cable|2023-04-09|
|       C003| Keyboard|2023-04-08|
+-----------+---------+----------+

